In [1]:
# !pip install transformers
!pip install -U nltk
!python -m spacy download en
!pip install bert-extractive-summarizer
!pip install spacy
!pip install -U pywsd
!pip install gensim
!pip install flask
!pip install flask-ngrok
!pip install pyngrok==4.1.1
!pip install pdfminer
!ngrok authtoken 2Lg2ySPqLXcz9sva55ZnKuTu8nN_24NA5AXW65LeWv3URHfb9

2023-11-03 01:17:55.988495: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-03 01:17:55.988558: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-03 01:17:55.988594: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-03 01:17:55.996711: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-03 01:17:57.222795: W tensorflow/c

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('popular')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data] 

True

##***Bert Summarizer***


In [3]:
!pip install git+https://github.com/boudinfl/pke.git
!pip install flashtext

  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-zbsqid9r
  Running command git clone --filter=blob:none --quiet https://github.com/boudinfl/pke.git /tmp/pip-req-build-zbsqid9r
  Resolved https://github.com/boudinfl/pke.git to commit 69871ffdb720b83df23684fea53ec8776fd87e63
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.0 MB/s eta 0:00:00
  Created wheel for pke: filename=pke-2.0.0-py3-none-any.whl size=6160628 sha256=083392fdcb80328ae22e117d39c5291c8a9e2765e1dad370cc1bc9833ce84e61
  Stored in directory: /tmp/pip-ephem-wheel-cache-phi7yoxr/wheels/8c/07/29/6b35bed2aa36e33d77ff3677eb716965ece4d2e56639ad0aab
Successfully built pke
  Preparing metadata (setup.py) ... done
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9297 sha256=8437a50840ddf4db96d8bc80632c9d70864787e1727603fc669787e4f394ce81
  Stored in directory: /root/.cache/pip/wheels/bc/be/39/c37ad168eb2ff644c9685f525544403721

In [4]:
from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

from summarizer import Summarizer

import pprint
import itertools
import re
import pke
import string
from nltk.corpus import stopwords

from nltk.tokenize import sent_tokenize
from flashtext import KeywordProcessor

import requests
import json
import re
import random
from pywsd.similarity import max_similarity
from pywsd.lesk import adapted_lesk
from pywsd.lesk import simple_lesk
from pywsd.lesk import cosine_lesk
from nltk.corpus import wordnet as wn

Warming up PyWSD (takes ~10 secs)... took 6.068053483963013 secs.


In [22]:
from flask import Flask, redirect, url_for, render_template, request, session, flash, jsonify
from flask_ngrok import run_with_ngrok

In [6]:
def keyPhrases(text):
    out=[]
    extractor = pke.unsupervised.MultipartiteRank()
    # load the content of the document, here document is expected to be a simple
    # test string and preprocessing is carried out using spacy
    stoplist = list(string.punctuation)  # List of punctuation marks
    stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']  # Additional symbols to exclude
    stoplist += stopwords.words('english')  # List of English stopwords from NLTK

    extractor.load_document(input=text, language='en', stoplist=stoplist, normalization=None)
    pos = {'PROPN'}  # Set containing the POS tag 'PROPN' for proper nouns

    extractor.candidate_selection(pos=pos)

    extractor.candidate_weighting(threshold=0.75, method='average')
    keyphrases = extractor.get_n_best(n=20)

    for key in keyphrases:
        out.append(key[0])

    return out

In [7]:
def tokenize_sentences(text):
    sentences = [sent_tokenize(text)]
    sentences = [y for x in sentences for y in x]
    # Remove any short sentences less than 20 letters.
    sentences = [sentence.strip() for sentence in sentences if len(sentence) > 20]
    return sentences

In [8]:
def get_sentences_for_keyword(keywords, sentences):
    keyword_processor = KeywordProcessor()
    keyword_sentences = {}
    for word in keywords:
        keyword_sentences[word] = []
        keyword_processor.add_keyword(word)
    for sentence in sentences:
        keywords_found = keyword_processor.extract_keywords(sentence)
        for key in keywords_found:
            keyword_sentences[key].append(sentence)

    for key in keyword_sentences.keys():
        values = keyword_sentences[key]
        values = sorted(values, key=len, reverse=True)
        keyword_sentences[key] = values
    return keyword_sentences

In [9]:
# Distractors from Wordnet
def get_distractors_wordnet(syn,word):
    distractors=[]
    word= word.lower()
    orig_word = word
    if len(word.split())>0:
        word = word.replace(" ","_")
    hypernym = syn.hypernyms()
    if len(hypernym) == 0:
        return distractors
    for item in hypernym[0].hyponyms():
        name = item.lemmas()[0].name()
        #print ("name ",name, " word",orig_word)
        if name == orig_word:
            continue
        name = name.replace("_"," ")
        name = " ".join(w.capitalize() for w in name.split())
        if name is not None and name not in distractors:
            distractors.append(name)
    return distractors

def get_wordsense(sent,word):
    word= word.lower()

    if len(word.split())>0:
        word = word.replace(" ","_")

    synsets = wn.synsets(word,'n')
    if synsets:
        wup = max_similarity(sent, word, 'wup', pos='n')
        adapted_lesk_output =  adapted_lesk(sent, word, pos='n')
        lowest_index = min (synsets.index(wup),synsets.index(adapted_lesk_output))
        return synsets[lowest_index]
    else:
        return None

# Distractors from http://conceptnet.io/
def get_distractors_conceptnet(word):
    word = word.lower()
    original_word= word
    if (len(word.split())>0):
        word = word.replace(" ","_")
    distractor_list = []
    url = "http://api.conceptnet.io/query?node=/c/en/%s/n&rel=/r/PartOf&start=/c/en/%s&limit=5"%(word,word)
    obj = requests.get(url).json()

    for edge in obj['edges']:
        link = edge['end']['term']

        url2 = "http://api.conceptnet.io/query?node=%s&rel=/r/PartOf&end=%s&limit=10"%(link,link)
        obj2 = requests.get(url2).json()
        for edge in obj2['edges']:
            word2 = edge['start']['label']
            if word2 not in distractor_list and original_word.lower() not in word2.lower():
                distractor_list.append(word2)

    return distractor_list

In [27]:
app = Flask(__name__, template_folder='/content/templates')
run_with_ngrok(app)

In [28]:
@app.route('/')
def home():
    return render_template("index.html")

@app.route('/saveData', methods=['POST'])
def saveData():
    output_string = StringIO()
    print(request.files)
    if 'testReport' in request.files:
        in_file= request.files['testReport']
        parser = PDFParser(in_file)
        doc = PDFDocument(parser)
        rsrcmgr = PDFResourceManager()
        device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.create_pages(doc):
            interpreter.process_page(page)

        fetch_string = ""
        fetch_string += output_string.getvalue()
        fetch_string = fetch_string.replace('\n', '')


        print("String From PDF:\n", fetch_string)

        model = Summarizer()
        result = model(fetch_string, min_length=60, max_length = 500, ratio = 0.4)

        summarized_text = ''.join(result)
        summarized_text = summarized_text.replace('\n', '')

        print(summarized_text)

        keywords = keyPhrases(fetch_string)
        print (keywords)
        filtered_keys=[]
        for keyword in keywords:
            if keyword.lower() in summarized_text.lower():
                filtered_keys.append(keyword)

        print (filtered_keys)

        sentences = tokenize_sentences(summarized_text)
        keyword_sentence_mapping = get_sentences_for_keyword(filtered_keys, sentences)

        print (keyword_sentence_mapping)

        key_distractor_list = {}

        for keyword in keyword_sentence_mapping:
            wordsense = get_wordsense(keyword_sentence_mapping[keyword][0],keyword)
            if wordsense:
                distractors = get_distractors_wordnet(wordsense,keyword)
                if len(distractors) ==0:
                    distractors = get_distractors_conceptnet(keyword)
                if len(distractors) != 0:
                    key_distractor_list[keyword] = distractors
            else:

                distractors = get_distractors_conceptnet(keyword)
                if len(distractors) != 0:
                    key_distractor_list[keyword] = distractors

        # print ("#############################################################################")
        # print ("Here is the Generated MCQ for the Provided Document")
        # print ("#############################################################################\n\n")

        mcq_data = []

        for each in key_distractor_list:
            sentence = keyword_sentence_mapping[each][0]
            pattern = re.compile(each, re.IGNORECASE)
            question_text = pattern.sub(" _______ ", sentence)

            choices = [each.capitalize()] + key_distractor_list[each]
            random.shuffle(choices)
            correct_answer = choices.index(each.capitalize())

            # Create a dictionary representing the MCQ question
            mcq_question = {
                "question_text": question_text,
                "choices": choices,
                "correct_answer": correct_answer
            }

            mcq_data.append(mcq_question)

        # Serialize the MCQ data to JSON
        json_mcq_data = json.dumps(mcq_data, indent=4)

        # Print or return the JSON data
        print(json_mcq_data)
        return jsonify(mcq_data)
    else:
        return 'Not done'

if __name__=='__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://07c2-34-90-89-211.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [03/Nov/2023 01:45:06] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Nov/2023 01:45:07] "GET /favicon.ico HTTP/1.1" 404 -


ImmutableMultiDict([('testReport', <FileStorage: 'test.pdf' ('application/pdf')>)])
String From PDF:
 The Greek historian knew what he was talking about. The Nile River fed Egyptian civilization for hundreds of years. The Longest River the Nile is 4,160 miles long—the world’s longest river. It begins near the equator in Africa and flows north to the Mediterranean Sea. In the south the Nile churns with cataracts. A cataract is a waterfall. Near the sea the Nile branches into a delta. A delta is an area near a river’s mouth where the water deposits fine soil called silt. In the delta, theNile divides into many streams. The river is called the upper Nile in the south and the lower Nile in the north. For centuries,heavy rains in Ethiopia caused the Nile to flood every summer. The floods deposited rich soil along the Nile’s shores. This soilwas fertile, which means it was good for growing crops. Unlike the Tigris and Euphrates, the Nile River flooded at the same timeevery year, so farmers c

...

[
    {
        "question_text": " _______  cut the stems into strips, pressed them, and dried theminto sheets that could be rolled into scrolls.",
        "choices": [
            "Xhosa",
            "Burundian",
            "Zimbabwean",
            "Nigerien",
            "Senegalese",
            "Djiboutian",
            "Sudanese",
            "Somalian",
            "Sierra Leonean",
            "Malawian",
            "Gabonese",
            "Tuareg",
            "Tanzanian",
            "Kenyan",
            "Black African",
            "Swazi",
            "Mozambican",
            "Zulu",
            "Angolan",
            "Malian",
            "Fulani",
            "Cameroonian",
            "Egyptian",
            "Nigerian",
            "Libyan",
            "Gambian",
            "Carthaginian",
            "Guinean",
            "Zairese",
            "Rwandan",
            "Basotho",
            "Ewe",
            "Togolese",
            "Namibian",
            "Ethio

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


The Nile River fed Egyptian civilization for hundreds of years. It begins near the equator in Africa and flows north to the Mediterranean Sea. Red Land, Black Land The ancient Egyptians lived in narrow bands of land on each side of the Nile. They called this region the black land because of the fertile soil that the floods deposited. The red land was the barren desert beyond the fertile region. Isolation Theharsh desert acted as a barrier to keep out enemies. Each year, Egyptian farmers watched for white birds called ibises, which flew up from the south. When the birds arrived, the annual flood waters would soon follow. Working together, they dug irrigation canals that carried river water to dry areas. Then they used a tool called a shaduf to spread the water across the fields. These innovative, or new, techniques gave them more farmland. Egyptian Crops Ancient Egyptians grew a large variety of foods. They were the first to grind wheat into flour and to mix the flour with yeast and wat

INFO:werkzeug:127.0.0.1 - - [03/Nov/2023 01:48:54] "POST /saveData HTTP/1.1" 200 -


[
    {
        "question_text": " _______  cut the stems into strips, pressed them, and dried theminto sheets that could be rolled into scrolls.",
        "choices": [
            "Zambian",
            "Senegalese",
            "Malawian",
            "Liberian",
            "Egyptians",
            "Namibian",
            "Sierra Leonean",
            "Tuareg",
            "Carthaginian",
            "Madagascan",
            "Algerian",
            "Guinean",
            "Djiboutian",
            "Fulani",
            "Eurafrican",
            "Xhosa",
            "Tunisian",
            "Tanzanian",
            "Moroccan",
            "Berber",
            "Ethiopian",
            "Zairese",
            "Congolese",
            "Zimbabwean",
            "Rwandan",
            "Sudanese",
            "Chadian",
            "Egyptian",
            "Gabonese",
            "Malian",
            "Libyan",
            "Nigerian",
            "Kenyan",
            "Basotho",
            